# Preprocesamiento de datos

## Ejercicios: Parte I

<div class="alert-warning">
    
Indicaciones: Los ejercicios 1, 2, 3 y 4 deben leer su data de links en Github y el código y los resultados deben ser publicados en páginas en Github.

</div>

## Ejercicio 1

<div class="alert-success">
    
1. Use two maps of points.

2. Compute the distance matrix for both maps.

3. Select one row of the distance matrix, and plot the two points with the minimal distance on top of the country of your choosing. (distancia entre punto a punto)
</div>

Mapas de puntos: Plantas de energía solar y eólica en China

Fuente: [Global Power Plant Database](https://datasets.wri.org/dataset/globalpowerplantdatabase)

In [52]:
import pandas as pd
import geopandas as gpd
import os

PowerPlants = pd.read_csv("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/data/global_power_plant_database.csv", dtype={'other_fuel3': 'object'})

China_PowerPlants = PowerPlants[PowerPlants["country"]=="CHN"]
China_PowerPlants = China_PowerPlants.loc[:,["name","gppd_idnr","capacity_mw","latitude","longitude","primary_fuel"]]
China_PowerPlants.reset_index(drop=True, inplace=True)

#Dataframe a GeoDataframe:
China_PowerPlants_gpd = gpd.GeoDataFrame(data=China_PowerPlants.copy(),
                                        geometry=gpd.points_from_xy(x=China_PowerPlants.longitude,
                                        y=China_PowerPlants.latitude),
                                        crs=4326)

#Cambiando los CRSs por unos proyectados y que coincidan con los de China3415:
China3415_PowerPlants = China_PowerPlants_gpd.to_crs(3415)

#Mapas de puntos: Plantas de energía solar y eólica
China3415_SolarPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Solar"]
China3415_SolarPowerPlants = China3415_SolarPowerPlants.rename(columns={"name":"SolarPowerPlant_Name"})
China3415_SolarPowerPlants = China3415_SolarPowerPlants.reset_index()
China3415_SolarPowerPlants.drop(columns="index",inplace=True)


China3415_WindPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Wind"]
China3415_WindPowerPlants = China3415_WindPowerPlants.rename(columns={"name":"WindPowerPlant_Name"})
China3415_WindPowerPlants = China3415_WindPowerPlants.reset_index()
China3415_WindPowerPlants.drop(columns="index",inplace=True)


# Geopackage con los 2 mapas de puntos:
China3415_SolarPowerPlants.to_file(os.path.join("maps","China_3415","China3415_Maps.gpkg"), layer="SolarPower_Plants", driver="GPKG",index="")
China3415_WindPowerPlants.to_file(os.path.join("maps","China_3415","China3415_Maps.gpkg"), layer="WindPower_Plants", driver="GPKG",index="")

## Ejercicio 2

<div class="alert-success">
    
1. Use a map of points and a map of lines.

2. Compute the distance matrix for both maps.

3. Select one line of the distance matrix, and plot the closests and the farthest point to that line. (Se puede seleccionar un río o un sistema y calcular el punto más cercano o lejano a la línea. Si se escoge un punto, se calcula la línea más cercana y lejana)
    
    
</div>

Plantas de energía nuclear en China:

In [53]:
China3415_NuclearPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Nuclear"]
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.rename(columns={"name":"NuclearPowerPlant_Name"})
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.reset_index()
China3415_NuclearPowerPlants.drop(columns="index",inplace=True)

China3415_NuclearPowerPlants.to_file(os.path.join("maps","China_3415","China3415_Maps.gpkg"), layer="NuclearPower_Plants", driver="GPKG",index="")

- Mapa de China: [World_Countries](https://www.efrainmaps.es/english-version/free-downloads/world/)

- Ríos en China: [World_Hydrography](https://www.efrainmaps.es/english-version/free-downloads/world/)

In [60]:
#Mapa de China
countries = gpd.read_file("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_Countries/World_Countries.shp")
China = countries[countries.COUNTRY=='China']
China3415_country = China.to_crs(3415)
China3415_country = China3415_country.reset_index()
China3415_country.drop(columns="index",inplace=True)
China3415_country.to_file(os.path.join("maps","China_3415","China3415_Maps.gpkg"), layer="Country", driver="GPKG",index="")

#Mapa de ríos en el mundo
Riverslink = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_Hydrography/World_Hydrography.shp"
Rivers = gpd.read_file(Riverslink)
Rivers3415 = Rivers.to_crs(3415)

#Recortar los ríos para que solo se encuentren los que encierran a China
China3415_Rivers = gpd.clip(gdf = Rivers3415, mask = China3415_country)
China3415_Rivers = China3415_Rivers.reset_index()
China3415_Rivers.drop(columns="index",inplace=True)
China3415_Rivers = China3415_Rivers.rename(columns={"NAME":"River_Name"})
China3415_Rivers = China3415_Rivers.rename(columns={"NAME":"River_Name", "SYSTEM":"System"})
China3415_Rivers.replace({None:"None"}, inplace = True)

China3415_Rivers.to_file(os.path.join("maps","China_3415","China3415_Maps.gpkg"), layer="Rivers", driver="GPKG",index="")

In [59]:
China3415 = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/China_3415/China3415_Maps.gpkg"
from fiona import listlayers
listlayers(China3415)

['SolarPower_Plants', 'WindPower_Plants', 'NuclearPower_Plants']

## Ejercicio 3

<div class="alert-success">
    
1. Create a set of points and a set of polygons

2. Compute the distance matrix for both sets.

3. Select one polygon of the distance matrix, and plot the closests and the farthest point to that polygon.
</div>   

Conjunto de polígonos: Convex hulls de ríos acorde al sistema al que pertenecen. Los que no forman parte de un sistema de ríos, se han engoblado en un mismo convex hull.

In [11]:
#GeoDataFrame, agrupando por ríos
China3415_RiverSystems = China3415_MajorRivers.dissolve(by="System")
China3415_RiverSystems = China3415_RiverSystems.drop(columns = ["River_Name","MILES","KILOMETERS"])
China3415_RiverSystems = China3415_RiverSystems.reset_index()
China3415_RiverSystems.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer='RiverSystems', driver="GPKG",index="")

#Convertir a convex hull y luego a un GeoDataFrame con polígonos
China3415_RiverSystems_hulls = China3415_RiverSystems.convex_hull.to_frame()
China3415_RiverSystems_hulls.rename(columns = {0:"geometry"}, inplace = True)
China3415_RiverSystems_hulls = China3415_RiverSystems_hulls.set_geometry("geometry")
China3415_RiverSystems_hulls = China3415_RiverSystems.reset_index()
China3415_RiverSystems_hulls

China3415_RiverSystems_hulls.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer='RiverSystems_hulls', driver="GPKG",index="")

Conjunto de puntos: Plantas de energía nuclear en China

## Ejercicio 4

<div class="alert-success">
    
1. Create a set of points and a set of lines

2. Get the buffer for the lines, select different values for the distance. Jugar con varias distancias, pero solo colocar una que tenga resultados interesantes en el notebook final.

3. Keep the points that are within the buffer (as in point 2, you need to play with differn distances until you show something interesting. Todos los puntos que están dentro del buffer creado para la línea.
   

</div>   

- Set de puntos: China3415_NuclearPowerPlants
- Set de líneas: China3415_RiverSystems